In [25]:
# Install huggingface hub if not already installed
# !pip install huggingface-hub

# Login to Hugging Face
# !huggingface-cli login

!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 13.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 13.2 MB/s eta 0:00:00a 0:00:01


In [9]:
# Install required packages
!pip install torch torchvision torchaudio
!pip install transformers datasets accelerate bitsandbytes
!pip install -q loralib
!pip install -q git+https://github.com/huggingface/peft.git

In [10]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model

In [11]:
# Check CUDA availability
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Current device: {torch.cuda.get_device_name()}")

# Configure quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

CUDA available: True
Current device: NVIDIA GeForce RTX 4090


In [ ]:
model_id = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

In [21]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)

model = get_peft_model(model, lora_config)

In [26]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# Load dataset
dataset = load_dataset("Amod/mental_health_counseling_conversations")

# Convert to pandas DataFrame for easier splitting
df = dataset['train'].to_pandas()

# Split into train/test (80/20 split)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert back to HuggingFace datasets
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Create a DatasetDict
from datasets import DatasetDict

dataset_split = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

print(f"Train set size: {len(train_dataset)}")
print(f"Test set size: {len(test_dataset)}")

Generating train split: 100%|██████████| 3512/3512 [00:00<00:00, 522688.09 examples/s]

Train set size: 2809
Test set size: 703


In [ ]:
# Replace 'dataset_name' with your specific dataset
dataset = load_dataset('dataset_name')
max_length = 512

def preprocess_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )

tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)